**1. Initial setup for selenium pakage with state links**

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import time
import pandas as pd

#driver setup
driver=webdriver.Chrome()
driver.get('https://www.redbus.in')
driver.maximize_window()
time.sleep(2)

#viewing all states
view_all_states=driver.find_element(By.XPATH,"//*[@id='homeV2-root']/div[3]/div[1]/div[2]/a")
view_all_states.click()

#due to jumping tab to fetch the current url to driver
current_url = view_all_states.get_attribute('href')
driver.get(current_url)

#all_links
state_links=driver.find_elements(By.CLASS_NAME,'D113_link')
state_links_dict={}
for links in state_links:
    state_links_dict.update({links.text : links.get_attribute('href')})

In [2]:
state_links_dict

{'RSRTC': 'https://www.redbus.in/online-booking/rajasthan-state-road-transport-corporation',
 'PEPSU (Punjab)': 'https://www.redbus.in/online-booking/pepsu-punjab',
 'Himachal Pradesh Tourism Development Corporation (HPTDC)': 'https://www.redbus.in/online-booking/himachal-pradesh-tourism-development-corporation-hptdc',
 'UPSRTC': 'https://www.redbus.in/online-booking/upsrtc',
 'Chandigarh Transport Undertaking (CTU)': 'https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu',
 'HRTC': 'https://www.redbus.in/online-booking/hrtc',
 'APSRTC': 'https://www.redbus.in/online-booking/apsrtc',
 'KSRTC (Kerala)': 'https://www.redbus.in/online-booking/ksrtc-kerala',
 'TNSTC': 'https://www.redbus.in/online-booking/tnstc',
 'Puducherry Road Transport Corporation (PRTC)': 'https://www.redbus.in/online-booking/puducherry-road-transport-corporation-prtc',
 'Kadamba Transport Corporation Limited (KTCL)': 'https://www.redbus.in/online-booking/ktcl',
 'TSRTC': 'https://www.redbus.in/on

**2. ALL BUS DETAILS COMMON FUNCTION**

In [5]:
def scrapper(State_link, State_Name):
    
    """
    This Function works for 
    
    1. Scrapping data from www.redbus.in website. 
    2. Convert all the data from dict to DataFrame.
    
    """
    
    # State bus data dictionary
    State_data_dict = {
        "State": [], "routeName": [], "routeLink": [], "busName": [], "busType": [], "departingTime": [],
        "Duration": [], "ReachingTime": [], "StarRating": [], "Price": [], "SeatAvailability": []
    }

    # Required packages
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    import time
    import pandas as pd
    import numpy as np

    # Initialize the driver
    driver = webdriver.Chrome()

    # Access the link from the dictionary
    driver.get(State_link)
    driver.maximize_window()
    time.sleep(3)

    # Pagination and route collection
    State_route_links_list = []
    wait = WebDriverWait(driver, 20)
    page_number = 0

    for i in range(1, 10):
        try:
            pagination_container = wait.until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='root']/div/div[4]/div[10]"))
            )
            next_page_button = pagination_container.find_element(
                By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"
            )
            action = ActionChains(driver)
            action.move_to_element(next_page_button).perform()
            time.sleep(2)
            next_page_button.click()
            page_number += 1
        except:
            pass

        # Collect route links
        State_route_links = driver.find_elements(By.XPATH, '//a[@class="route"]')
        for route in State_route_links:
            State_route_links_list.append(route.get_attribute('href'))

    # Get unique route links
    State_unique_route_links = list(set(State_route_links_list))
    print(len(State_unique_route_links))

    # Iterate through all route links to get bus info
    for route_link in State_unique_route_links:
        driver.get(route_link)
        time.sleep(3)

        # Click "View Bus" for additional buses
        try:
            other_busses = driver.find_elements(By.XPATH, '//div[@class="button"]')
            for i in reversed(other_busses):
                i.click()
                time.sleep(2)
        except:
            pass
        
        #Using java script for scroll page for infinite scroll
        try:
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                # Scroll down to the bottom
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait for the new page section to load
                time.sleep(3)

                # Calculate new scroll height and compare with the last scroll height
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break  # If the heights are the same, we’ve reached the bottom
                last_height = new_height
        except:
            print('error occurred')

        # Scrape bus data
        data = driver.find_elements(By.XPATH, "//div[@class='clearfix bus-item-details']")
        State_route_name = driver.find_element(By.XPATH, '//h1[@class="D136_h1"]').text
        State_route_link = route_link

        # Push data to dictionary
        for bus in data:
            State_data_dict['State'].append(f'{State_Name}')
            State_data_dict['routeName'].append(State_route_name)
            State_data_dict['routeLink'].append(State_route_link)
            State_data_dict['busName'].append(bus.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text)
            State_data_dict['busType'].append(bus.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text)
            State_data_dict['departingTime'].append(bus.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text)
            State_data_dict['Duration'].append(bus.find_element(By.XPATH, ".//div[@class='column-four p-right-10 w-10 fl']").text)
            State_data_dict['ReachingTime'].append(bus.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text)
            try:
                State_data_dict['StarRating'].append(float(bus.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text))
            except:
                State_data_dict['StarRating'].append(0)
            try:
                State_data_dict['Price'].append(bus.find_element(By.XPATH, ".//span[@class='f-19 f-bold']").text)
            except:
                State_data_dict['Price'].append(0)
            try:
                State_data_dict['SeatAvailability'].append(bus.find_element(By.XPATH, ".//div[@class='seat-left m-top-16']").text[0:2])
            except:
                State_data_dict['SeatAvailability'].append(0)
            
    # Convert data into DataFrame
    df = pd.DataFrame(State_data_dict)

    # Convert data types
    df['State'] = df['State'].astype(str)
    df['routeName'] = df['routeName'].astype(str)
    df['routeLink'] = df['routeLink'].astype(str)
    df['busName'] = df['busName'].astype(str)
    df['busType'] = df['busType'].astype(str)
    df['Duration'] = df['Duration'].astype(str)
    df['StarRating'] = df['StarRating'].astype(float)
    df['SeatAvailability'] = df['SeatAvailability'].astype(int)

    # Replace empty strings with NaN in the Price column
    df['Price'] = df['Price'].replace('', np.nan)

    # Convert Price column to numeric type, ignoring errors (to handle any invalid values)
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

    # Fill NaN values with 0 (or any other strategy, depending on your needs)
    df['Price'].fillna(0)

    # Now convert the Price column to float
    df['Price'] = df['Price'].astype(float)


    # Clean time string function
    def clean_time_string(time_str):
        time_str = time_str.strip()  # Remove any surrounding spaces
        if len(time_str) <= 2:  # If time only has hours, add ":00" for minutes
            return time_str + ':00'
        return time_str

    # Apply the cleaning function before converting to datetime
    df['departingTime'] = df['departingTime'].apply(clean_time_string)
    df['ReachingTime'] = df['ReachingTime'].apply(clean_time_string)
    
    # Convert times to datetime objects
    df['departingTime'] = pd.to_datetime(df['departingTime'], format='%H:%M').dt.time
    df['ReachingTime'] = pd.to_datetime(df['ReachingTime'], format='%H:%M').dt.time

    return df

STATE LIST

1. RSRTC
2. PEPSU
3. UPSRTC
4. HRTC
5. South Bengal State Transport Corporation (SBSTC)
6. NORTH BENGAL STATE TRANSPORT CORPORATION
7. WBTC (CTC)
8. APSRTC
9. TSRTC
10. Sikkim Nationalised Transport (SNT)


In [6]:
#RSRTC
df1=scrapper(state_links_dict['RSRTC'],"RSRTC")
time.sleep(3)

18


In [9]:
#PEPSU
df2=scrapper(state_links_dict['PEPSU (Punjab)'],"PEPSU (Punjab)")
time.sleep(3)

18


In [11]:
#UPSRTC
df3=scrapper(state_links_dict['UPSRTC'],"UPSRTC")
time.sleep(3)

49


In [13]:
#HRTC
df4=scrapper(state_links_dict["HRTC"],"HRTC")
time.sleep(3)

35


In [15]:
#South Bengal State Transport Corporation (SBSTC)
df5=scrapper(state_links_dict['South Bengal State Transport Corporation (SBSTC)'],'South Bengal State Transport Corporation (SBSTC)')
time.sleep(3)

41


In [17]:
#NORTH BENGAL STATE TRANSPORT CORPORATION
df6=scrapper(state_links_dict['NORTH BENGAL STATE TRANSPORT CORPORATION'],'NORTH BENGAL STATE TRANSPORT CORPORATION')
time.sleep(3)

47


In [19]:
#WBTC (CTC)
df7=scrapper(state_links_dict['WBTC (CTC)'],'WBTC (CTC)')
time.sleep(3)

38


In [21]:
#APSRTC
df8=scrapper(state_links_dict['APSRTC'],'APSRTC')
time.sleep(3)

46


In [24]:
#TSRTC
df9=scrapper(state_links_dict['TSRTC'],'TSRTC')
time.sleep(3)

26


In [26]:
#KSRTC
df10=scrapper(state_links_dict['KSRTC (Kerala)'],'KSRTC (Kerala)')
time.sleep(3)

15


In [28]:
#concat all the dataframe
concat_df=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10],axis=0,ignore_index=True)

In [29]:
import pandas as pd #for additional backup of scrapped data
concat_df.to_csv('redbus.csv',index=False)

In [1]:
import pandas as pd
df=pd.read_csv('redbus.csv')
df.shape

(15569, 11)

In [3]:
#Sql part for data pushing

#Sql connection
import mysql.connector
connection = mysql.connector.connect(
host="localhost",
user="root",
password="",
database='redbus')
cursor = connection.cursor(buffered=True)

#Migrate all data into SQL
query_for_data_push = """INSERT INTO REDBUS (State, routeName, routeLink, busName, busType, departingTime, Duration, ReachingTime, StarRating, Price, SeatAvailability) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
for index,rows in df.iterrows():
    cursor.execute(query_for_data_push,(rows['State'],rows['routeName'],rows['routeLink'],rows['busName'],rows['busType'],rows['departingTime'],rows['Duration'],rows['ReachingTime'],rows['StarRating'],rows['Price'],rows['SeatAvailability']))
connection.commit()

In [2]:
#truncate table for add new data
#Sql connection
import mysql.connector
connection = mysql.connector.connect(
host="localhost",
user="root",
password="",
database='redbus')
cursor = connection.cursor(buffered=True)
cursor.execute("TRUNCATE redbus;")